# Creating Dataclass

In [15]:
from pydantic.dataclasses import dataclass
from typing import Tuple
from enum import Enum

In [16]:
@dataclass
class IceCreamMix:
    name: str
    flavor: str
    toppings: Tuple[str, ...]
    scoops: int

In [17]:
ice_cream_mix = IceCreamMix("PB&J", "vanilla", ("strawberries", "sprinkles"), 2)
ice_cream_mix

IceCreamMix(name='PB&J', flavor='vanilla', toppings=('strawberries', 'sprinkles'), scoops=2)

# Restrict Attributes using Enum

In [18]:
class Flavor(str, Enum):
    chocolate = 'chocolate'
    vanilla = 'vanilla'
    strawberry = 'strawberry'
    mint = 'mint'
    coffee = 'coffee'
    peanut_butter = 'peanut butter'
    
class Topping(str, Enum):
    sprinkles = 'sprinkles'
    cookies = 'cookies'
    brownie = 'brownie'
    strawberries = 'strawberries'
    hot_fudge = 'hot fudge'
    whipped_cream = 'whipped cream'
    

In [19]:
@dataclass
class IceCreamMix2:
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int

In [22]:
ice_cream_mix = IceCreamMix2("PB&J", Flavor.vanilla, (Topping.cookies, Topping.sprinkles), 2)
ice_cream_mix

IceCreamMix2(name='PB&J', flavor=<Flavor.vanilla: 'vanilla'>, toppings=(<Topping.cookies: 'cookies'>, <Topping.sprinkles: 'sprinkles'>), scoops=2)

If user tries to select different flavor than those provided, pydantic throws validation error.

In [23]:
ice_cream_mix = IceCreamMix2("PB&J", 'blood', (Topping.cookies, Topping.sprinkles), 2)
ice_cream_mix

ValidationError: 1 validation error for IceCreamMix2
flavor
  value is not a valid enumeration member; permitted: 'chocolate', 'vanilla', 'strawberry', 'mint', 'coffee', 'peanut butter' (type=type_error.enum; enum_values=[<Flavor.chocolate: 'chocolate'>, <Flavor.vanilla: 'vanilla'>, <Flavor.strawberry: 'strawberry'>, <Flavor.mint: 'mint'>, <Flavor.coffee: 'coffee'>, <Flavor.peanut_butter: 'peanut butter'>])

In [24]:
# lets try to pass #scoops as string
ice_cream_mix = IceCreamMix2("PB&J", Flavor.mint, (Topping.cookies, Topping.sprinkles), '2')
ice_cream_mix
# it will be converted to int as pydantic supports type coersion

IceCreamMix2(name='PB&J', flavor=<Flavor.mint: 'mint'>, toppings=(<Topping.cookies: 'cookies'>, <Topping.sprinkles: 'sprinkles'>), scoops=2)

# Extending capabilities using BaseModel
 - serialization
 - First class JSON support

In [25]:
from pydantic import BaseModel

In [26]:
class IceCreamMix3(BaseModel):
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int

In [29]:
# BaseModel requires keyword arguments
ice_cream_mix = IceCreamMix3(name = "PB&J", 
                             flavor = Flavor.mint, 
                             toppings = (Topping.cookies, Topping.sprinkles), 
                             scoops = '2')
ice_cream_mix

IceCreamMix3(name='PB&J', flavor=<Flavor.mint: 'mint'>, toppings=(<Topping.cookies: 'cookies'>, <Topping.sprinkles: 'sprinkles'>), scoops=2)

In [30]:
ice_cream_mix.json()

'{"name": "PB&J", "flavor": "mint", "toppings": ["cookies", "sprinkles"], "scoops": 2}'

In [31]:
# create object from JSON
ice_cream_mix_json = IceCreamMix3.parse_raw('{"name": "PB&J", "flavor": "mint", "toppings": ["cookies", "sprinkles"], "scoops": 2}')
ice_cream_mix_json

IceCreamMix3(name='PB&J', flavor=<Flavor.mint: 'mint'>, toppings=(<Topping.cookies: 'cookies'>, <Topping.sprinkles: 'sprinkles'>), scoops=2)

In [34]:
from pydantic import ValidationError
# print error as JSON
try:
    ice_cream_mix = IceCreamMix3(name = "PB&J", 
                                 flavor = 'blood', 
                                 toppings = (Topping.cookies, Topping.sprinkles), 
                                 scoops = '2')
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "flavor"
    ],
    "msg": "value is not a valid enumeration member; permitted: 'chocolate', 'vanilla', 'strawberry', 'mint', 'coffee', 'peanut butter'",
    "type": "type_error.enum",
    "ctx": {
      "enum_values": [
        "chocolate",
        "vanilla",
        "strawberry",
        "mint",
        "coffee",
        "peanut butter"
      ]
    }
  }
]


# Restricting field values

In [36]:
from pydantic import Field

class IceCreamMix4(BaseModel):
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int = Field(..., gt=0, lt=5)  # ... means required field, greater than 0 and less than 5

In [42]:
try:
    ice_cream_mix = IceCreamMix4(name = "PB&J", 
                                 flavor = Flavor.vanilla, 
                                 toppings = (Topping.cookies, Topping.sprinkles), 
                                 scoops = 5)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "scoops"
    ],
    "msg": "ensure this value is less than 5",
    "type": "value_error.number.not_lt",
    "ctx": {
      "limit_value": 5
    }
  }
]


# Adding Custom Validators

In [43]:
from pydantic import validator

In [44]:
class IceCreamMix5(BaseModel):
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int = Field(..., gt=0, lt=5)
        
    @validator('toppings')
    def check_toppings(cls, toppings):
        """Ensure that there aren't too many toppings"""
        if len(toppings) > 3:
            raise ValueError('Too many toppings')
        return toppings

In [46]:
try:
    ice_cream_mix = IceCreamMix5(name = "PB&J", 
                                 flavor = Flavor.vanilla, 
                                 toppings = (Topping.cookies, Topping.sprinkles, Topping.brownie, Topping.hot_fudge), 
                                 scoops = 3)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "toppings"
    ],
    "msg": "Too many toppings",
    "type": "value_error"
  }
]


# Using root_validator to validate complete object
Let's assume that we aren't allowed to use cookies topping with mint flavor.

In [48]:
from pydantic import root_validator

class IceCreamMix6(BaseModel):
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int = Field(..., gt=0, lt=5)
        
    @validator('toppings')
    def check_toppings(cls, toppings):
        """Ensure that there aren't too many toppings"""
        if len(toppings) > 3:
            raise ValueError('Too many toppings')
        return toppings
    
    @root_validator
    def check_topping_and_flavor(cls, values):
        """Ensure cookies topping isn't used with mint flavor"""
        flavor = values.get('flavor')
        toppings = values.get('toppings')
        if flavor == Flavor.mint and Topping.cookies in toppings:
            raise ValueError("Mint flavor can't be used with cookies topping.")
        return values

In [49]:
try:
    ice_cream_mix = IceCreamMix6(name = "PB&J", 
                                 flavor = Flavor.mint, 
                                 toppings = (Topping.cookies, Topping.sprinkles), 
                                 scoops = 3)
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "__root__"
    ],
    "msg": "Mint flavor can't be used with cookies topping.",
    "type": "value_error"
  }
]
